## Stock Returns of the FOMC Cycle

Preamble: Scientific findings are by definition reproducible by controlled experiment. This notebook validates results in this paper: http://faculty.haas.berkeley.edu/vissing/cycle_paper_cieslak_morse_vissingjorgensen.pdf

#### 1. Web Scrape FOMC Meeting End Dates

In [2]:
import requests
from lxml import html
from datetime import *
import pandas as pd

Recent Meetings: https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm

In [2]:
url = 'https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm'
fomccalendars = html.fromstring(requests.get(url).content)

In [3]:
fomc_dates = []
for panel in fomccalendars.xpath(".//div[@class='panel panel-default']"):
    year = int(panel.xpath(".//div[@class='panel-heading']//text()")[0][0:4])
    months = panel.xpath(".//div[contains(@class, 'month')]//text()")
    dates = panel.xpath(".//div[contains(@class, 'date')]/text()")
    m = [ datetime.strptime(month.split('/')[-1][0:3], '%b').month for month in months ]
    d = [ int(day.split('-')[-1].rstrip('*')) for day in dates if not "unscheduled" in day ]
    fomc_dates += [ datetime(year, m, d).date() for m,d in zip(m, d) ]    

Historical Meetings: https://www.federalreserve.gov/monetarypolicy/fomc_historical_year.htm

In [118]:
url = 'https://www.federalreserve.gov/monetarypolicy/fomc_historical_year.htm'
fomc_historical_year = html.fromstring(requests.get(url).content)
fomchistorical = [ 'https://www.federalreserve.gov'+href for href in fomc_historical_year.xpath(".//div[@id='article']//a/@href") ]

In [5]:
dates = []
for url in fomchistorical:
    historical = html.fromstring(requests.get(url).content)
    dates += historical.xpath(".//h5/text()")

In [6]:
meetings = [ meeting.split() for meeting in dates if "Meeting" in meeting ]
years = [ int(meeting[-1]) for meeting in meetings]
months = [ datetime.strptime(date[0], '%B').month if len(date)==5 else datetime.strptime(date[0], '%B').month+1 for date in meetings ]
days = [ int(date[1].split('-')[-1]) if len(date)==5 else 1 for date in meetings ]
fomc_dates += [ datetime(y, m, d).date() for y,m,d in zip(years, months, days) ]

In [13]:
fomc_dates.sort()
fomc_dates.remove(datetime(2003, 9, 15).date())
fomc_df = pd.DataFrame(fomc_dates, columns=['fomc_date'], dtype='datetime64[ns]')
fomc_df.to_csv('fomc_dates.csv', index=False)

#### 2. Calculate FOMC Calendar

In [82]:
fomc_load = pd.read_csv('fomc_dates.csv', parse_dates=[0], infer_datetime_format=True)
fomc_dates = [ timestamp.date() for timestamp in fomc_load['fomc_date'].tolist() ] 

In [ ]:
from bisect import *
import numpy as np

In [19]:
calendar = pd.bdate_range(min(fomc_dates), max(fomc_dates))
last_fomc = [ fomc_dates[bisect_right(fomc_dates, date.date())-1] for date in calendar ]
next_fomc = [ fomc_dates[bisect_left(fomc_dates, date.date())] for date in calendar ]
days_since = [ np.busday_count(last_fomc[d], calendar[d]) for d in range(len(calendar)) ]
days_before = [ -np.busday_count(calendar[d], next_fomc[d]) for d in range(len(calendar)) ]
fomc_day = [ days_before[d] if days_before[d] >= -6 else days_since[d] for d in range(len(calendar)) ]
fomc_week = [ (fomc_day[d] + 1) // 5 for d in range(len(calendar))]

In [63]:
fomc_calendar = pd.DataFrame(data={'fomc_day': fomc_day, 'fomc_week': fomc_week}, index=calendar)
fomc_calendar.index.name = 'date'
fomc_calendar.to_csv('fomc_calendar.csv')

#### 3. Calculate 5-Day Returns

In [3]:
fomc_calendar = pd.read_csv('fomc_calendar.csv', index_col=0, parse_dates=[0], infer_datetime_format=True)

Stocks Daily: http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html

In [4]:
url = 'http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip'
factors = pd.read_csv(url, index_col=0, engine='python', skiprows=4, skipfooter=2, parse_dates=[0], infer_datetime_format=True, compression='zip') 

In [113]:
market = factors.drop(['SMB', 'HML', 'RF'], 1).asfreq(freq='B', fill_value=0)
market['bus_week'] = [ ((market['Mkt-RF'][i:i+5]/100+1).prod()-1)*100 for i in range(len(market)) ] 
fomc_final = fomc_calendar.join(market).drop(['Mkt-RF'], 1).dropna()

In [114]:
fomc_final.to_csv('fomc_final.csv')

#### 4. Plot Figure 1

In [97]:
fomc_final = pd.read_csv('fomc_final.csv', index_col=0, parse_dates=[0], infer_datetime_format=True)

In [115]:
cycle = fomc_final['1994':]
fomc = cycle[cycle['fomc_day'] < 34]
group = fomc.groupby('fomc_day')

In [105]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [120]:
source = ColumnDataSource(group)

hover = HoverTool(tooltips=[("(x,y)", "(@fomc_day, $y{0.00%})")], mode='vline')
p = figure(title='Stock returns of the FOMC cycle, 1994-2018', plot_width=800, plot_height=400) 
p.add_tools(hover)

p.line(x='fomc_day', y='bus_week_mean', source=source)
p.circle(x='fomc_day', y='bus_week_mean', source=source)
p.xaxis.axis_label = 'Days since FOMC meeting (weekends excluded)'
p.yaxis.axis_label = 'Avg. 5-day stock return, t to t+4'
labels = LabelSet(x='fomc_day', y='bus_week_mean', text='fomc_day', source=source, x_offset=5, y_offset=-10, text_font_size='10pt')
p.add_layout(labels)

show(p)